In [1]:
import librosa
import matplotlib.pyplot as plt
import IPython.display as dis
import numpy as np
import os, sys
from tqdm import tqdm
import pandas as pd
import librosa.display

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
if not os.path.isdir('/content/melspectrogram_data'):
    os.makedirs('/content/melspectrogram_data')

In [4]:
!unzip /content/gdrive/MyDrive/final/melspectrogram_data_plus.zip -d /content/melspectrogram_data

Archive:  /content/gdrive/MyDrive/final/melspectrogram_data_plus.zip
  inflating: /content/melspectrogram_data/1002914.npy  
  inflating: /content/melspectrogram_data/10124.npy  
  inflating: /content/melspectrogram_data/101529890.npy  
  inflating: /content/melspectrogram_data/101545093.npy  
  inflating: /content/melspectrogram_data/1016369.npy  
  inflating: /content/melspectrogram_data/101678251.npy  
  inflating: /content/melspectrogram_data/101721111.npy  
  inflating: /content/melspectrogram_data/1017622.npy  
  inflating: /content/melspectrogram_data/1023889.npy  
  inflating: /content/melspectrogram_data/10291.npy  
  inflating: /content/melspectrogram_data/102979321.npy  
  inflating: /content/melspectrogram_data/104184.npy  
  inflating: /content/melspectrogram_data/105014.npy  
  inflating: /content/melspectrogram_data/105187.npy  
  inflating: /content/melspectrogram_data/1055513.npy  
  inflating: /content/melspectrogram_data/105699.npy  
  inflating: /content/melspectrog

In [ ]:
if not os.path.isdir('/content/mfcc_data'):
    os.makedirs('/content/mfcc_data')

In [ ]:
!unzip -o /content/gdrive/MyDrive/hindi/mfcc_data.zip -d /content/mfcc_data

In [5]:
rawdata = pd.read_csv('/content/gdrive/MyDrive/final/newtag_1118.csv')

In [15]:
lst = os.listdir('/content/melspectrogram_data')
lst[0]

'5663983.npy'

In [21]:
rawdata

,Unnamed: 0,IDX,song_id,song_title,대분류,중분류
0,0,57088.0,4203068.0,소녀,차분한,평온한
1,1,258244.0,4203787.0,다알아,밝은,희망찬
2,2,57038.0,4209541.0,기억속의 먼 그대에게,우울한,슬픈
3,3,156375.0,4214654.0,이별하면,우울한,외로운
4,4,56764.0,4230004.0,I'm Fine Thank You,우울한,슬픈
...,...,...,...,...,...,...
3288,3288,20349965.0,34850634.0,사랑인가 봐,밝은,발랄한 희망찬
3289,3289,20350574.0,34890162.0,다시 만날까 봐,우울한,슬픈 외로운
3290,3290,20351090.0,34927767.0,정이라고 하자 (Feat. 10CM),차분한,잔잔한 평온한
3291,3291,20351312.0,34943312.0,나의 X에게,밝은,발랄한


In [23]:
lst = os.listdir('/content/melspectrogram_data')
data_x = []
data_y = []
errors = []
for i in tqdm(range(len(rawdata))):
    try:
        data_x.append(np.load('/content/melspectrogram_data/' + str(int(rawdata['song_id'].iloc[i]))+'.npy'))
        data_y.append(rawdata['대분류'].iloc[i])
    except:
        errors.append(str(int(rawdata['song_id'].iloc[i]))+'.npy')


100%|██████████| 3293/3293 [00:31<00:00, 105.78it/s]


In [26]:
len(data_y)

3253

In [ ]:
rawdata.iloc[111]

Unnamed: 0         149
song_id       30829905
new_tag            신나는
Name: 111, dtype: object

In [27]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(data_y)

In [28]:
wav_to_img_data = []
target = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(data_x))):
    for i in range(20):
        if 128 * (i+1) > len(data_x[idx][0]):
            break
        wav_to_img_data.append(data_x[idx][:, 128* i : 128 * (i+1)].reshape(128,-1,1))
        target.append(data_y[idx])
wav_to_img_data = np.array(wav_to_img_data)
target = np.array(target)

100%|██████████| 3253/3253 [00:00<00:00, 32803.53it/s]


In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, BatchNormalization, Activation, Flatten

def VGG_model():
    
    inputs = Input(shape=(128,128,1))
    x = Conv2D(64,3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation='softmax')(x)
    
    return tf.keras.Model(inputs, x)

In [30]:
model = VGG_model()

In [ ]:
model.summary()

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy',  metrics=['acc'])

In [ ]:
history = model.fit(wav_to_img_data, target, epochs=10, batch_size=32, validation_split=0.2)

In [157]:
model.save('/content/gdrive/MyDrive/final/vgg_1118.h5')

### 추론

In [37]:
test_data = pd.read_csv('/content/gdrive/MyDrive/final/test_newtag_1118.csv')

test_x = []
test_y = []
for i in tqdm(range(len(test_data))):
    try:
        test_x.append(np.load('/content/melspectrogram_data/' + str(int(test_data['song_id'].iloc[i]))+'.npy'))
        test_y.append(test_data['대분류'].iloc[i])
    except:
        pass

100%|██████████| 480/480 [00:12<00:00, 38.82it/s]


In [38]:
test_y = label_encoder.fit_transform(test_y)

In [158]:
test_whole = []
for i in range(len(test_x)):
    one_sample = []
    for j in range(len(test_x[i][0])):
        if 32*j+128 >= len(test_x[i][0]):
            break
        one_sample.append(test_x[i][:, 32*j : 32*j+128].reshape(128,128,-1))
    test_whole.append(one_sample)

In [159]:
cnt = 0
correct = 0
total_result = []
co_lst = []

for i in tqdm(range(len(test_whole))):
    infer = model.predict(np.array(test_whole[i]))
    infer = np.argmax(infer, axis=1)
    count_ = np.unique(infer, return_counts=True)
    total_result.append(count_)
    result = count_[0][np.argmax(count_[1])]
    cnt += 1
    # print(count_)
    # print(result, test_y[i])
    if result == test_y[i]:
        co_lst.append(i)
        correct += 1

  0%|          | 0/480 [00:00<?, ?it/s]

11/11 [==============================] - 1s 59ms/step


  0%|          | 1/480 [00:00<05:21,  1.49it/s]

14/14 [==============================] - 1s 50ms/step


  0%|          | 2/480 [00:01<05:36,  1.42it/s]

...

100%|█████████▉| 478/480 [04:52<00:01,  1.65it/s]

9/9 [==============================] - 0s 57ms/step


100%|█████████▉| 479/480 [04:53<00:00,  1.69it/s]

10/10 [==============================] - 1s 56ms/step


100%|██████████| 480/480 [04:53<00:00,  1.63it/s]


In [160]:
total_result

[(array([1, 2, 4, 5]), array([  3,   1, 183, 137])),
 (array([1, 2, 3, 4, 5]), array([  1,  21,   4, 312,  83])),
 (array([1, 3, 4, 5]), array([  3,   2, 104, 228])),
 (array([0, 1, 2, 3, 4, 5]), array([ 15,  17,  57, 118,  58,  84])),
 (array([0, 1, 2, 3, 4, 5]), array([ 27,  50,   3, 110,  23,  31])),
 (array([0, 1, 2, 3, 4, 5]), array([  2,   2,  41,  11, 167,  85])),
 (array([0, 1, 2, 3, 4, 5]), array([  3,   3,   7,   1, 158, 136])),
 (array([0, 1, 2, 3, 4, 5]), array([41, 88,  7, 91,  8, 56])),
 (array([0, 1, 2, 3, 4, 5]), array([  6,  65,   4,  25, 117, 127])),
 (array([0, 1, 2, 3, 4, 5]), array([  1,   6,   3,   5, 259,  50])),
 (array([1, 4, 5]), array([  1, 189, 139])),
 (array([2, 4, 5]), array([  7, 306,  70])),
 (array([0, 1, 2, 3, 4, 5]), array([  2,  54,  11,  77,  38, 131])),
 (array([4, 5]), array([102, 214])),
 (array([0, 1, 2, 3, 4, 5]), array([  6,   5,   1,  14, 255,  69])),
 (array([4, 5]), array([235, 119])),
 (array([1, 4, 5]), array([  5, 178, 155])),
 (array([

In [151]:
# i = 0
# temp = np.argmax(model.predict(np.array(test_whole[i])) , axis=1)
# cnts = np.unique(temp, return_counts=True)
c_n_t = 0
co = 0
el = 0
for i in range(len(total_result)):
    cnts = total_result[i]
    tsum = np.sum(cnts[1])
    topval = cnts[1][(np.argmax(cnts[1]))]
    
    if (cnts[1][np.argsort(cnts[1])[-1]] - cnts[1][np.argsort(cnts[1])[-2]]) / tsum > 0.4:
        # print((cnts[1][np.argsort(cnts[1])[-1]] - cnts[1][np.argsort(cnts[1])[-2]]))
        if cnts[0][(np.argmax(cnts[1]))] == test_y[i]:
            co += 1
        # print(cnts[0][(np.argmax(cnts[1]))])
        # print(test_y[i])
        # print(cnts)
    else:
        el += 1
        if cnts[0][np.argsort(cnts[1])[-2]] == test_y[i]:
            co += 1
        elif cnts[0][(np.argmax(cnts[1]))] == test_y[i]:
            co += 1
       
    c_n_t += 1

In [ ]:
444,447

In [155]:
c_n_t - el

96

In [153]:
el

384

In [152]:
co/c_n_t

0.61875

In [ ]:
np.array(total_result)[co_lst]

False

5

In [53]:
label_encoder.classes_

array(['공격적인', '긴장감', '밝은', '신나는', '우울한', '차분한'], dtype='<U4')

In [ ]:
ddddd = np.load('/content/melspectrogram_data/' + '460396'+'.npy')

In [ ]:
music = []
# for idx in tqdm(range(len(data_x))):
idx = 111
for i in range(len(data_x[idx][0])):
    if i+128 == len(data_x[idx][0]):
        break
    music.append(data_x[idx][:, i : 128 + i].reshape(128,128,-1))

In [ ]:
music = []
for i in range(len(ddddd[0])):
    if i+128 == len(ddddd[0]):
        break
    music.append(ddddd[:, i : 128 + i].reshape(128,128,-1))

In [ ]:
music = []
for i in range(len(ddddd[0])):
    if 128*(i+1) >= len(ddddd[0]):
        break
    music.append(ddddd[:, 128*i : 128 * (1+ i)].reshape(128,128,-1))

In [ ]:
result = model.predict(np.array(music))

4/4 [==============================] - 1s 257ms/step


In [ ]:
res = np.argmax(result, axis=1)

In [ ]:
np.unique(res, return_counts=True)

(array([0, 2, 3, 4, 5]), array([23, 16, 16, 27, 15]))

In [ ]:
label_encoder.classes_

array(['공격적인', '긴장감', '밝은', '신나는', '우울한', '차분한'], dtype='<U4')

In [ ]:
data_y[111]

3

In [ ]:
np.unique(rawdata['senti_tag'],return_counts=True)

(array(['불안', '슬픔', '신남', '편안해요', '행복해요', '화나요'], dtype=object),
 array([200, 200, 200, 384, 151, 202]))

In [ ]:
rawdata[620:650]

,song_id,song_name,song_lyric,senti_tag
620,31246870,한(一),Do you remember you remember\nremember what yo...,행복해요
621,3628521,4AM,일어난 건지 \n아직 잠들지 못한 건지\n당장이 내일인지 오늘인지 \n어젠지도 모르...,행복해요
622,30689468,결혼,결혼에 대하여 \n예쁜 단어를 골라 \n예쁜 칭찬을 하고\n예쁜 밤을 만들 것\n결...,행복해요
623,32183386,Blueming,‘뭐해?‘라는 두 글자에\n‘네가 보고 싶어’ 나의 속마음을 담아 우\n이모티콘 하...,행복해요
624,3772006,Love Love Echo Song,No lyric,행복해요
625,30677447,Who's Got You Singing Again,It happened so easy at the start\nGot too used...,행복해요
626,32823088,On and On,I never thought I had a chance\nYou seemed so ...,행복해요
627,30677444,Futures,Wind on the cool grey water \nVoices way back ...,행복해요
628,31048045,Me & You ◑,One small step for man \none giant leap for ma...,행복해요
629,31263803,I Might ◑,Say hello to Tokyo\nI can see myself living ri...,행복해요


In [ ]:
lst = os.listdir('/content/mfcc_data')
datam_x = []
datam_y = []
for i in tqdm(range(len(rawdata))):
   datam_x.append(np.load('/content/mfcc_data/' + str(rawdata['song_id'].iloc[i])+'.npy'))
   datam_y.append(rawdata['senti_tag'].iloc[i])

100%|██████████| 1337/1337 [00:02<00:00, 632.95it/s]


In [ ]:
datam_x[0].shape

(50, 10231)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
datam_y = label_encoder.fit_transform(datam_y)

In [ ]:
dda = []
tt = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(datam_x))):
    for i in range(20):
        if 100 * (i+1) > len(datam_x[idx][0]):
            break
        dda.append(datam_x[idx][:, 100* i : 100 * (i+1)].reshape(50,100,1))
        tt.append(datam_y[idx])
dda = np.array(dda)
tt = np.array(tt)

100%|██████████| 1337/1337 [00:00<00:00, 35145.30it/s]


In [ ]:
dda[0].shape

(50, 256, 1)

In [ ]:
!nvidia-smi

Tue Nov 15 06:45:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |  14632MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, BatchNormalization, Activation, Flatten

def mfcc():
    
    inputs = Input(shape=(50,100,1))
    x = Conv2D(64,3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512,3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2)(x)
    
    # x = Conv2D(512,3, padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Conv2D(512,3, padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Conv2D(512,3, padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = MaxPooling2D(2)(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='softmax')(x)
    
    return tf.keras.Model(inputs, x)

In [ ]:
mfcc_model = mfcc()

In [ ]:
mfcc_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 50, 100, 1)]      0         
                                                                 
 conv2d_74 (Conv2D)          (None, 50, 100, 64)       640       
                                                                 
 batch_normalization_74 (Bat  (None, 50, 100, 64)      256       
 chNormalization)                                                
                                                                 
 activation_74 (Activation)  (None, 50, 100, 64)       0         
                                                                 
 conv2d_75 (Conv2D)          (None, 50, 100, 64)       36928     
                                                                 
 batch_normalization_75 (Bat  (None, 50, 100, 64)      256       
 chNormalization)                                          

In [ ]:
mfcc_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy',  metrics=['acc'])

In [ ]:
history_ = mfcc_model.fit(dda, tt, epochs=2, batch_size=32, validation_split=0.4)

Epoch 1/2
502/502 [==============================] - 44s 89ms/step - loss: 0.1180 - acc: 0.9546 - val_loss: 3.6125 - val_acc: 0.5026
Epoch 2/2
502/502 [==============================] - 43s 85ms/step - loss: 0.1211 - acc: 0.9526 - val_loss: 4.2967 - val_acc: 0.3862


In [ ]:
mfcc_model.evaluate(dda, tt)

836/836 [==============================] - 18s 22ms/step - loss: 1.8442 - acc: 0.7092


[1.8442332744598389, 0.7091622948646545]